In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np 
import os
import pandas as pd
from gamescout_db import db, cur
import time

In [2]:
data = pd.read_sql('select * from BatterStats;', con=db).dropna()

data["GOT_HIT"] = [1 if x > 0 else 0 for x in data["HITS"]]
data["NOT_HIT"] = [1 if x == 0 else 1 for x in data["HITS"]]

y = data[["GOT_HIT", "NOT_HIT"]].astype(np.float32)
x = data[['1_AGO_AVG', '2_AGO_AVG', '3_AGO_AVG', '4_AGO_AVG', '5_AGO_AVG']].astype(np.float32)

In [3]:
train_x = x[:60000].as_matrix()
test_x = x[60001:77750].as_matrix()

train_y = y[:60000].as_matrix()
test_y = y[60001:77750].as_matrix()

In [4]:
numFeatures = train_x.shape[1]
numLabels = train_y.shape[1]
numEpochs = 27000
learningRate = tf.train.exponential_decay(  learning_rate = .0008, 
                                            global_step = 1, 
                                            decay_steps = train_x.shape[0], 
                                            decay_rate = 0.95, 
                                            staircase = True)

X = tf.placeholder(tf.float32, [None, numFeatures])
yGold = tf.placeholder(tf.float32, [None, numLabels])

In [5]:
weights = tf.Variable(  tf.random_normal([numFeatures, numLabels], 
                            mean = 0, 
                            stddev = (np.sqrt(6 / numFeatures + numLabels + 1)), 
                            name = "weights"))

bias = tf.Variable(  tf.random_normal([1, numLabels], 
                            mean = 0, 
                            stddev = (np.sqrt(6 / numFeatures + numLabels + 1)), 
                            name = "bias"))

In [6]:
init_OP = tf.global_variables_initializer()
apply_weights_OP = tf.matmul(X, weights, name = "apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name = "add_bias")
activation_OP = tf.nn.sigmoid(add_bias_OP, name = "activation")
cost_OP = tf.nn.softmax_cross_entropy_with_logits(activation_OP-yGold, labels=yGold, name="squared_error_cost")
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

# Create a tensorflow session
sess = tf.Session()

# Initialize all tensorflow variables
sess.run(init_OP)

# argmax(activation_OP, 1) gives the label our model thought was most likely
# argmax(yGold, 1) is the correct label
correct_predictions_OP = tf.equal(activation_OP, yGold)
# False is 0 and True is 1, what was our average?
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))
# Summary op for accuracy
accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

In [7]:
# Training epochs
for i in range(numEpochs):
    step = sess.run(training_OP, feed_dict={X: train_x, yGold: train_y})
    
    if i % 5000 == 0:
        pred = sess.run(
            [tf.reduce_mean(activation_OP)], 
            feed_dict={X: test_x, yGold: test_y})
        
        print(pred)
        time.sleep(0.25)

[nan]
[nan]
[nan]
[nan]
[nan]
[nan]


In [9]:
correct_prediction = tf.equal(activation_OP, yGold)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={X: test_x, yGold: test_y}))

0.140181
